In [16]:
import pandas as pd
import os

testcase_root = r'/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20'

testcase_names = [filename for filename in os.listdir(testcase_root) if filename.startswith('C') and filename.endswith('.csv')]
print(testcase_names)
testcase_paths = [os.path.join(testcase_root, testcase_name) for testcase_name in sorted(testcase_names)]
print(testcase_paths)

['C7_testcase_cooking_method.csv', 'C1_testcase_breakfast_freq4_familarity5.csv', 'C2_testcase_lunch_and_dinner_freq5_familarity5.csv', 'C3_testcase_memory_freq3_familarity4.csv', 'C6_testcase_ingrdient_freq.csv', 'C5_testcase_main_food_freq4_familarity5.csv', 'C4_testcase_infer_freq5_familarity5.csv']
['/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/C1_testcase_breakfast_freq4_familarity5.csv', '/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/C2_testcase_lunch_and_dinner_freq5_familarity5.csv', '/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/C3_testcase_memory_freq3_familarity4.csv', '/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/C4_testcase_infer_freq5_familarity5.csv', '/date0/crwu/zjts_foodname/data/classification/outputs_by_type_food_name_all_V100R25C20/C5_testcase_main_food_freq4_familarity5.csv', '/date0/crwu/zjts_foodnam

## 1. 最最主要的问题就是就是我昨天下午开会时提到的，数据全部来自于下厨房的开源数据集，因此很多菜品的命名极不规范，随意性很大，因此如果决定用这个数据源作为测试数据集的源头数据，那么选出来的每一道菜名，都需要人工确认，大多数菜名，可能都是要调整，否则由于命名的随意性，导致后期大模型评估时可能会有较大的偏差。（比如蔬菜包，水果汁，包子馒头花卷，家庭油条，小油条，咖啡牛奶，早餐三明治，大白馒头，小笼包子，白菜肉包子，半荤素包子，猪肉白菜馅包子，瘦肉皮蛋粥，简易面包，冰豆浆，水波蛋，鲜奶，碧绿青菜包，超绵吐司，黄豆豆浆，油炸鬼，奶馒头等）
## 2. 像水煮蛋，五香蛋这样非常简单的菜，以不同名字出现了很多次，必要性不大（熟鸡蛋，水煮蛋，煮茶蛋，糖心鸡蛋，煮茶叶蛋，茶叶蛋，酱油卤蛋，五香茶叶蛋，五香鸡蛋，煎鸡蛋，鸡蛋煎，爱心煎蛋，芝士煎蛋，腌鸡蛋，酱油荷包蛋，培根煎蛋，单面煎蛋，酱油卤蛋等）。
## 3. 较多地出现了咸菜和酱料，目前看对碳水和热量等影响不是太大，也许后续对盐分等控制需要考虑（五香萝卜干，腐乳馒头，萝卜咸菜，紫葡萄果酱，辣腐乳，辣菜丝，干咸菜，酱菜，腌小菜，芥菜咸菜，酱黄豆等）。

# 总结
## 1.命名不规范，菜名不典型，需要修复为建议菜名
## 2.重复度高的名字需要聚类
## 3.酱料、咸菜、腌菜（可能还有其他的）要减少

In [23]:
df = pd.read_csv(testcase_paths[0])
df.insert(1, '建议菜名', [pd.NA for _ in range(len(df))])
food_names = df.sample(300)['菜品名'].tolist()
print(",".join(food_names))
# print(f'请判断下列菜品是否存在过多不必要的修饰词，如：大肉包子应该改为肉包子\n{",".join(food_names)}')

红薯玉米粥,简易面包,生煎馒头,香菇包子,烧饼,萝卜咸菜,牛奶燕麦,广式葱油饼,素包子,葱香饼,鸡蛋土司,金枪鱼三明治,牛奶馒头,包子,葱花千层饼,红糖馒头,五谷粥,大饼,葱香肉松卷,超绵吐司,白吐司面包,红豆八宝粥,炒馒头丁,猪肉大葱包子,菠菜鸡蛋粥,发面包子,奶香面包,芝士吐司,牛奶燕麦片,鸡蛋韭菜包子,花卷馒头,东北煎饼,老式油条,金牛角面包,奶馒头,香肠面包,韭菜鸡蛋馅包子,脆麦片,果酱面包,葱香肉饼,杂粮馒头,牛角面包,原味豆浆,芥菜疙瘩咸菜,小黄米粥,早餐三明治,土豆饼,葱蛋饼,甜烧饼,牛角包,小笼包,馒头花卷,小笼包子,炸油条,蜂蜜酸奶,老面馒头,酱黄瓜,黑米粥,水波蛋,大白馒头,红糖全麦馒头,葱油发面饼,鸡蛋葱花饼,韭菜鸡蛋盒子,三明治,全麦切片面包,肠仔卷,豆腐脑,酱菜,生煎,烫面葱油饼,早餐煎饼,大肉包,鸡蛋煎,猪肉粥,青菜香菇包子,酱黄豆,麦包,桃子酸奶,瘦肉皮蛋粥,全麦土司,葱油酥饼,白糖烧饼,五谷杂粮粥,芝士鸡蛋卷,干咸菜,土司片,猪油菜包,发面糖饼,葱花烙饼,葡萄果酱,杂粮包,起酥面包,芝士煎蛋,红豆餐包,肉末粥,辣腐乳,超熟鲜奶吐司,发面葱油饼,油条,腌鸡蛋,咖啡牛奶,卤肉包子,海苔饭团,奶香吐司面包,葱花面包,醇豆浆,炒馒头,绿豆面煎饼果子,千层饼,肉松,培根煎蛋,煮茶蛋,鸡蛋薄饼,白菜肉包子,葱香火腿面包,豆浆油条,肠仔小面包,煮茶叶蛋,煮荷包蛋,猪肉芹菜馅饼,萝卜包子,芝麻鸡蛋饼,韭菜蛋饼,鸡肉三明治,香菇瘦肉粥,虾肉小馄饨,全脂纯牛奶,炒蛋,香菇菜包,玉米粥,葱香蛋饼,芝麻饼,单面煎蛋,香葱肉松面包,猪肉芸豆包子,猪肉大白菜包子,奶香麦片粥,豆馅饼,荷包蛋,腌小菜,猪肉豆角大包,煎吐司,紫葡萄果酱,馅饼,腐乳馒头,小米红薯粥,内脂豆腐脑,小米南瓜粥,酱油卤蛋,辣菜丝,五谷杂粮豆浆,豆沙小馒头,青菜香菇包,酱肉包,奶香面包片,大米绿豆粥,全麦面包片,酱油荷包蛋,绿豆小米粥,爱心煎蛋,红枣燕麦粥,红烧肉包子,豆包,大米粥,碧绿青菜包,包子馒头花卷,圆白菜包子,家庭油条,火腿三明治,麦片粥,奶粉酸奶,开花馒头,红豆沙包,小包子,地瓜粥,葱烧饼,白水煮蛋,糖包子,猪肉包子,糖心鸡蛋,菜肉粥,豆沙餐包,黑米红豆粥,杭州小笼包,熟鸡蛋,营养八宝粥,香菇青菜包,芥菜咸菜,蓝莓酸奶,牛奶燕麦粥,蔬菜包,煎蛋三明治,稀饭,卷心菜包子,水果汁,早餐

太好了 👍
以下是为你系统整理的一份「**菜名修饰词删/保留判定表（标准化命名规则）**」，可直接作为食品名称清洗或菜品分类的参考准则，用于自动化或人工审核。

---

# 🍱 菜名修饰词删与保留判定表

---

## 🥇 一、尺寸类修饰词 → ✅ **可删**

**作用**：仅描述体积、份量，不改变食物类型。
**建议**：统一删除。

| 示例   | 标准化结果    | 说明        |
| ---- | -------- | --------- |
| 大白馒头 | 白馒头 / 馒头 | “大”为体积修饰。 |
| 小油条  | 油条       | “小”仅表示尺寸。 |
| 迷你包子 | 包子       | 无实质差异。    |
| 超大烧饼 | 烧饼       | 尺寸描述。     |

**规则**：`if 修饰词 ∈ {大, 小, 迷你, 超大, 特大, mini}` → 删除。

---

## 🥈 二、场景/时段类修饰词 → ✅ **可删**

**作用**：描述食用场景或时间，与食物类型无关。
**建议**：删除。

| 示例   | 标准化结果 |
| ---- | ----- |
| 早餐煎饼 | 煎饼    |
| 晚餐粥  | 粥     |
| 家常包子 | 包子    |
| 简易面包 | 面包    |

**规则**：`if 修饰词 ∈ {早餐, 午餐, 晚餐, 家常, 快手, 简易}` → 删除。

---

## 🥉 三、工艺类修饰词 → ✅ **多数可删**

**作用**：说明制作方式；若该方式为常规方法，可删。若是独特风格或决定风味的关键，则保留。

| 示例           | 处理方式                       | 原因 |
| ------------ | -------------------------- | -- |
| 发面红糖饼 → 红糖饼  | ✅ 常规发酵工艺                   |    |
| 烫面葱油饼 → 葱油饼  | ✅ 工艺常规化                    |    |
| 烤烧饼 → 烧饼     | ✅ 烤为默认工艺                   |    |
| 煎馒头片 → 保留“煎” | ⚠️ “煎馒头片”不同于“馒头片”，工艺决定成品类型 |    |
| 炸酱面 → 保留“炸”  | ⚠️ 炸酱定义风味核心                |    |

**判断逻辑：**

* 若工艺仅是**常规制作方式** → 删。
* 若工艺决定**菜品类型或口味核心** → 保留。

---

## 🌿 四、风味/香气类修饰词 → ⚠️ **慎删**

**作用**：体现主要风味差异。
**建议**：若仅为营销形容词（如“香”“醇”），可删；若影响风味识别（如“葱香”“奶香”“麻辣”），应保留。

| 示例           | 是否删除         | 说明 |
| ------------ | ------------ | -- |
| 香酥油条 → 油条    | ✅ “香酥”是口感描述。 |    |
| 奶香面包 → 面包    | ✅ 营销词。       |    |
| 葱香肉松卷 → 保留葱香 | ❌ 风味变化明显。    |    |
| 麻辣豆腐 → 保留麻辣  | ❌ 风味特征。      |    |

**规则建议**：

* `if 修饰词 ∈ {香, 香酥, 醇, 纯, 原味}` → 可删。
* `if 修饰词 ∈ {葱香, 奶香, 麻辣, 咸香, 甜辣, 酸甜}` → 保留。

---

## 🍡 五、颜色/外观类修饰词 → ⚠️ **可删或保留视情况**

**作用**：仅描述色泽或外观。
**建议**：若颜色是主要区分（如“红豆馒头”），保留；若仅为视觉形容（如“大白馒头”），可删。

| 示例                | 建议         | 说明 |
| ----------------- | ---------- | -- |
| 白馒头 → 保留或作为基础类    | “白”常用于基础款。 |    |
| 大白馒头 → 删“大”，保留“白” | 尺寸可删。      |    |
| 紫葡萄果酱 → 删“紫”      | 颜色冗余。      |    |
| 红豆八宝粥 → 保留“红豆”    | 主料差异。      |    |

---

## 🧂 六、结构性赘述 → ✅ **应删**

**作用**：语法或重复词缀。
**建议**：统一去除，保持命名规范化。

| 示例       | 标准化结果     |
| -------- | --------- |
| 小笼包子     | 小笼包       |
| 白菜猪肉馅的包子 | 白菜猪肉包子    |
| 吐司切片     | 吐司片 / 面包片 |
| 芝麻鸡蛋饼    | 鸡蛋饼       |

---

## 🍱 七、组合或混写型名称 → ⚠️ **应拆分**

**作用**：多个菜品混写，应分成独立记录。

| 示例     | 处理方式            |
| ------ | --------------- |
| 包子馒头花卷 | 拆分为包子 / 馒头 / 花卷 |
| 油条豆浆   | 拆分为油条 / 豆浆      |

---

## 🍞 八、非主料点缀类 → ⚠️ **视上下文而定**

**说明**：若点缀为装饰性撒料（芝麻、葱花），可删；若已形成固定组合或风味差异，应保留。

| 示例          | 建议    | 说明 |
| ----------- | ----- | -- |
| 芝麻鸡蛋饼 → 鸡蛋饼 | 撒料可删。 |    |
| 葱花饼 → 保留葱花  | 风味关键。 |    |
| 葱香肉饼 → 保留葱香 | 明显风味。 |    |

---

# ✅ 九、决策逻辑总表（适合自动清洗使用）

| 分类  | 删除建议 | 代表词汇    | 特别说明         |
| --- | ---- | ------- | ------------ |
| 尺寸类 | 删除   | 大、小、迷你  | 无风味差异        |
| 场景类 | 删除   | 早餐、家常   | 无食品差异        |
| 工艺类 | 多数删除 | 发面、烫面、烤 | 若工艺定义菜品类型则保留 |
| 风味类 | 慎删   | 香、葱香、奶香 | 若决定风味则保留     |
| 颜色类 | 视情况  | 白、紫、红   | 主料色差则保留      |
| 结构类 | 删除   | 子、馅的、切片 | 语法冗余         |
| 组合类 | 拆分   | /、与、及   | 一条记录一菜品      |
| 点缀类 | 视情况  | 芝麻、葱花   | 非主料可删        |

---

是否希望我帮你基于这套规则生成一个
📊 **Python自动菜名清洗脚本**（输入菜名列表 → 输出标准化结果、删除类别、原因说明、保留建议）？
可直接生成 CSV/Excel 文件，用于后续批量处理或模型训练数据清理。


## prompt:
根据上述清8条洗规则，请处理以下菜名：
红薯玉米粥,简易面包,生煎馒头,香菇包子,烧饼,萝卜咸菜,牛奶燕麦,广式葱油饼,素包子,葱香饼,鸡蛋土司,金枪鱼三明治,牛奶馒头,包子,葱花千层饼,红糖馒头,五谷粥,大饼,葱香肉松卷,超绵吐司,白吐司面包,红豆八宝粥,炒馒头丁,猪肉大葱包子,菠菜鸡蛋粥,发面包子,奶香面包,芝士吐司,牛奶燕麦片,鸡蛋韭菜包子,花卷馒头,东北煎饼,老式油条,金牛角面包,奶馒头,香肠面包,韭菜鸡蛋馅包子,脆麦片,果酱面包,葱香肉饼,杂粮馒头,牛角面包,原味豆浆,芥菜疙瘩咸菜,小黄米粥,早餐三明治,土豆饼,葱蛋饼,甜烧饼,牛角包,小笼包,馒头花卷,小笼包子,炸油条,蜂蜜酸奶,老面馒头,酱黄瓜,黑米粥,水波蛋,大白馒头,红糖全麦馒头,葱油发面饼,鸡蛋葱花饼,韭菜鸡蛋盒子,三明治,全麦切片面包,肠仔卷,豆腐脑,酱菜,生煎,烫面葱油饼,早餐煎饼,大肉包,鸡蛋煎,猪肉粥,青菜香菇包子,酱黄豆,麦包,桃子酸奶,瘦肉皮蛋粥,全麦土司,葱油酥饼,白糖烧饼,五谷杂粮粥,芝士鸡蛋卷,干咸菜,土司片,猪油菜包,发面糖饼,葱花烙饼,葡萄果酱,杂粮包,起酥面包,芝士煎蛋,红豆餐包,肉末粥,辣腐乳,超熟鲜奶吐司,发面葱油饼,油条,腌鸡蛋,咖啡牛奶,卤肉包子,海苔饭团,奶香吐司面包,葱花面包,醇豆浆,炒馒头,绿豆面煎饼果子,千层饼,肉松,培根煎蛋,煮茶蛋,鸡蛋薄饼,白菜肉包子,葱香火腿面包,豆浆油条,肠仔小面包,煮茶叶蛋,煮荷包蛋,猪肉芹菜馅饼,萝卜包子,芝麻鸡蛋饼,韭菜蛋饼,鸡肉三明治,香菇瘦肉粥,虾肉小馄饨,全脂纯牛奶,炒蛋,香菇菜包,玉米粥,葱香蛋饼,芝麻饼,单面煎蛋,香葱肉松面包,猪肉芸豆包子,猪肉大白菜包子,奶香麦片粥,豆馅饼,荷包蛋,腌小菜,猪肉豆角大包,煎吐司,紫葡萄果酱,馅饼,腐乳馒头,小米红薯粥,内脂豆腐脑,小米南瓜粥,酱油卤蛋,辣菜丝,五谷杂粮豆浆,豆沙小馒头,青菜香菇包,酱肉包,奶香面包片,大米绿豆粥,全麦面包片,酱油荷包蛋,绿豆小米粥,爱心煎蛋,红枣燕麦粥,红烧肉包子,豆包,大米粥,碧绿青菜包,包子馒头花卷,圆白菜包子,家庭油条,火腿三明治,麦片粥,奶粉酸奶,开花馒头,红豆沙包,小包子,地瓜粥,葱烧饼,白水煮蛋,糖包子,猪肉包子,糖心鸡蛋,菜肉粥,豆沙餐包,黑米红豆粥,杭州小笼包,熟鸡蛋,营养八宝粥,香菇青菜包,芥菜咸菜,蓝莓酸奶,牛奶燕麦粥,蔬菜包,煎蛋三明治,稀饭,卷心菜包子,水果汁,早餐粥,蛋煎饼,葱香薄饼,鮮奶,发面红糖饼,吐司切片,青菜皮蛋粥,烙煎饼,大葱肉包,火龙果酸奶,火腿鸡蛋面,煮番薯,炒蛋吐司,葱油煎饼,红萝卜鸡蛋饼,葱油蛋饼,巧克力牛奶,黄豆豆浆,鲜肉包,发酵乳,糙米粥,葱油肉饼,花生酱面包,猪肉香菇包,香酥油条,大葱肉饼,红豆黑米粥,五香鸡蛋,鸡蛋葱油饼,火腿面包,小油条,奶油小面包,蔬菜粥,葱香烧饼,冰豆浆,煎饼果子,芝麻煎饼,火腿鸡蛋饼,发面葱花饼,半荤素包子,早餐手抓饼,豆浆,火腿煎蛋,葱花手抓饼,肠仔包,蛋煎馒头片,菜包子,猪肉芹菜包子,小米粥,茶叶蛋,羊角面包,手抓饼,小薄饼,紫菜三角饭团,鸡蛋火腿三明治,煎蛋卷,大肉包子,红枣红糖馒头,香煎午餐肉,葱花卷,烤面包,豆沙馅包子,肉包,菜饼,葱油花卷,三鲜包子,豆沙毛毛虫面包,白菜猪肉包子,奶油面包,摊煎饼,葱花油饼,五香茶鸡蛋,千层葱油饼,鸡蛋卷饼,咸稀饭,芒果酸奶,香肠包,豆角猪肉包子,葱香鸡蛋煎饼,小米红枣粥,香酥鸡蛋灌饼,小土司,大果粒酸奶,葱油鸡蛋饼,草莓酱面包,白菜豆腐水煎包,特浓酸奶,葱花馒头,黄油吐司,白菜肉馅包子,五香萝卜干,猪肉白菜馅包子,葱花鸡蛋饼,油炸鬼,鸡蛋粥,四川咸菜,水煮蛋,馒头片,烤烧饼,白菜猪肉馅的包子,鸡蛋煎饼,煎鸡蛋,牛奶小馒头,白菜素馅包子

要求输出格式为csv格式，具体如下：
菜品名,删除原因,人工复核,清洗后菜品名
大白馒头,尺寸类,否,白馒头
芝麻鸡蛋饼,点缀类,是,鸡蛋饼

清洗原则：
1、输出清洗后的建议菜名
2、删除建议为建议删除的，人工复核类型为FALSE；否则为TRUE
3、删除原因填写删除原则：尺寸类/场景类/工艺类/风味类/颜色类/结构类/组合类/点缀类

分类	删除建议	代表词汇	特别说明
尺寸类	删除	大、小、迷你	无风味差异
场景类	删除	早餐、家常	无食品差异
工艺类	多数删除	发面、烫面、烤	若工艺定义菜品类型则保留
风味类	慎删	香、葱香、奶香	若决定风味则保留
颜色类	视情况	白、紫、红	主料色差则保留
结构类	删除	子、馅的、切片	语法冗余
组合类	拆分	/、与、及	一条记录一菜品
点缀类	视情况	芝麻、葱花	非主料可删

In [27]:
df = pd.read_csv(testcase_paths[1])
print(f'共有{len(df)}条数据')
df.insert(1, '建议菜名', [pd.NA for _ in range(len(df))])
food_names = df.sample(100)['菜品名'].tolist()
print(",".join(food_names))
# print(f'请判断下列菜品是否存在过多不必要的修饰词，如：大肉包子应该改为肉包子\n{",".join(food_names)}')

共有500条数据
西兰花炒肉,青椒牛柳,炖蛋,肉末白菜,蘑菇炒鸡肉,葱香豆腐,白菜炒肉丝,手撕白菜,醋溜手撕包菜,包菜炒饭,炒长豆角,海带炖肉,三鲜面,红烧鸡腿,猪肉白菜香菇饺子,泡菜,西红柿汤面,火腿炒鸡蛋,肉末炖蛋,红烧茄子,菠菜炖豆腐,火腿肠炒鸡蛋,油炸土豆条,炒花菜,土豆炒肉,葱花煎蛋,油菜炒肉,韭菜肉丝,韭菜馄饨,生煎饺子,青椒牛肉丝,炒肉,洋葱炒肉丝,韭菜炒肉丝,菠菜鸡蛋汤,炒鸡肉片,花菜肉片,蛋炒面,西红柿紫菜蛋汤,酱油炒饭,肉丝青椒,荠菜猪肉水饺,猪肉馅饺子,猪肉芹菜水饺,猪肉韭菜馅水饺,青菜香菇,素炒青菜,青椒炒鸡肉,猪肉胡萝卜饺子,鲜虾炒面,拌黄瓜,炒虾仁,西红柿青菜蛋汤,猪肉大葱烫面蒸饺,爆炒青菜,紫菜虾皮汤,肉片炒芹菜,青豆虾仁,西红柿盖浇饭,清炒生菜,青菜汤,腌菜,奥尔良鸡腿堡,拌面条,紫菜虾皮蛋花汤,炒豆腐干,奥尔良蜂蜜烤翅,爆炒鸡丁,清炒包菜,鸡蛋韭菜,蒜茸油麦菜,小炒牛肉,包菜炒五花肉,清炒小白菜,红烧大白菜,清炖鸡,肉蒸蛋,西红柿炒鸡蛋,小炒青菜,辣酱面,木耳炒肉丝,香菇鸡蛋汤,青椒干子肉丝,冬瓜排骨汤,素炒白菜丝,排骨莲藕汤,白菜炒蛋,红辣椒炒肉,韭菜鸡蛋面,鸡蛋豆腐汤,猪骨老火汤,炒酸辣白菜,胡萝卜丝炒鸡蛋,面包三明治,土豆烧鸡腿,懒人拌面,炒鸡片,盒饭,酱炒鸡蛋,排骨玉米汤


In [52]:
import random
print(testcase_names)
random.seed(5)
superviser = ['crwu', 'sylu'] * 4+ ['ylzhou', 'xkzhang', 'czguo'] * 2
C5 = ['ylzhou', 'sylu']
for name in C5:
    superviser.remove(name)
C1 = ['crwu'] + random.sample(superviser, 1)
for name in C1:
    superviser.remove(name)
C2 = random.sample(superviser, 2)
for name in C2:
    superviser.remove(name)
C3 = random.sample(superviser, 2)
for name in C3:
    superviser.remove(name)
C4 = random.sample(superviser, 2)
for name in C4:
    superviser.remove(name)
C6 = random.sample(superviser, 2)
for name in C6:
    superviser.remove(name)
C7 = random.sample(superviser, 2)
for name in C7:
    superviser.remove(name)
print(C1)
print(C2)
print(C3)
print(C4)
print(C5)
print(C6)
print(C7)


['C7_testcase_cooking_method.csv', 'C1_testcase_breakfast_freq4_familarity5.csv', 'C2_testcase_lunch_and_dinner_freq5_familarity5.csv', 'C3_testcase_memory_freq3_familarity4.csv', 'C6_testcase_ingrdient_freq.csv', 'C5_testcase_main_food_freq4_familarity5.csv', 'C4_testcase_infer_freq5_familarity5.csv']
['crwu', 'ylzhou']
['crwu', 'sylu']
['crwu', 'xkzhang']
['czguo', 'crwu']
['ylzhou', 'sylu']
['sylu', 'czguo']
['sylu', 'xkzhang']
